In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt

from datetime import date
from datetime import datetime
import calendar
import os
import re


from scipy import stats
from sklearn.neighbors import KernelDensity
import matplotlib.colors as mcolors
from matplotlib.animation import FuncAnimation, PillowWriter

import seaborn as sns
from pathlib import Path
from tqdm import tqdm_notebook, trange, tqdm
from PIL import Image, ImageDraw
import io
import random as r

from matplotlib.widgets import Slider, Button, RadioButtons
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing
from sklearn.pipeline import make_pipeline
# For downsampling
from sklearn.utils import resample,shuffle

In [2]:
df_companies = pd.read_csv("./data/companies.csv")
df_payments = pd.read_csv("./data/payments.csv")
df_physicians = pd.read_csv("./data/physicians.csv")

c:\python38\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (26,27,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
df_companies.rename({"State":"State_Comp","Country":"Country_Comp"}, axis='columns', inplace=True)
df_physicians.rename({"State":"State_Phys","Country":"Country_Phys"}, axis='columns', inplace=True)
pd.set_option('display.max_columns', None)
df = pd.merge(df_payments, df_physicians, left_on='Physician_ID', right_on='id', how='left').drop('id', axis=1)
df = pd.merge(df, df_companies, on='Company_ID', how='left')
df.rename({"Name":"Company_Name"}, axis='columns', inplace=True)


###test safekeeping dataframe
df_main_test=df[df.set=="test"]

In [39]:
df_scrap=df.query('Product_Name_1.notnull() and Product_Name_2.notnull()')
df_scrap['index_col'] = df_scrap.index
df_scrap_1=pd.wide_to_long(df_scrap, stubnames='Product_Name_', i=['index_col'],j='Product_Name_Number').reset_index()
df_scrap.drop(columns=['index_col'],inplace=True)
df_scrap_1['index_col'] = df_scrap_1.index
df_scrap_1=pd.wide_to_long(df_scrap_1, stubnames='Product_Type_', i=['index_col'],j='Product_Type_Number').reset_index()

df_scrap_1

# df_scrap.Ownership_Indicator.value_counts()
# df.Ownership_Indicator.value_counts()

<ipython-input-39-62a4ce39fed3>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_scrap['index_col'] = df_scrap.index
c:\python38\lib\site-packages\pandas\core\frame.py:4167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,index_col,Product_Type_Number,Physician_ID,Third_Party_Covered,Last_Name,Related_Product_Indicator,Name_Suffix,License_State_5,Ownership_Indicator,Product_Category_2,Product_Category_1,Company_ID,Primary_Specialty,State_Phys,License_State_4,State_Comp,Zipcode,Province,Contextual_Information,Product_Code_1,Product_Category_3,City,City_of_Travel,License_State_2,Number_of_Payments,State_of_Travel,Date,Country_Phys,Third_Party_Recipient,Nature_of_Payment_or_Transfer_of_Value,License_State_1,Country_Comp,Charity,Record_ID,Country_of_Travel,Product_Code_3,Product_Name_,set,Product_Code_2,First_Name,License_State_3,Company_Name,Middle_Name,Form_of_Payment_or_Transfer_of_Value,Product_Name_Number,Total_Amount_of_Payment_USDollars,Product_Type_
0,0,1,1965,NaN,HOUSE,Covered,NaN,NaN,No,NaN,NaN,10,Allopathic & Osteopathic Physicians|Family Med...,IN,NaN,IN,46038-2617,NaN,NaN,0002322930,NaN,FISHERS,NaN,NaN,1,NaN,08/01/2013,UNITED STATES,No Third Party Payment,Food and Beverage,IN,United States,No,17,NaN,NaN,STRATTERA,train,0002197590,BRET,NaN,"LILLY USA, LLC",A,In-kind items and services,1,8.92,Drug or Biological
1,1,1,3827,NaN,Mengesha,Covered,NaN,NaN,No,NaN,NaN,114,NaN,AL,NaN,MA,35801-5177,NaN,NaN,NaN,NaN,HUNTSVILLE,NaN,KY,1,NaN,08/01/2013,UNITED STATES,No Third Party Payment,Food and Beverage,AL,United States,No,24,NaN,NaN,TECFIDERA,train,NaN,Theodros,NaN,Biogen Idec Inc.,NaN,Cash or cash equivalent,1,10.14,Drug or Biological
2,2,1,4127,NaN,ROSENBLATT,Covered,NaN,NaN,No,NaN,NaN,242,Allopathic & Osteopathic Physicians|Internal M...,CT,NaN,IL,06051-3916,NaN,NaN,6476462530,NaN,NEW BRITAIN,NaN,NaN,1,NaN,08/01/2013,UNITED STATES,No Third Party Payment,Food and Beverage,CT,United States,No,28,NaN,6476433560,Nesina,train,6476425103,JOSEPH,NaN,"Takeda Pharmaceuticals America, Inc.",NaN,In-kind items and services,1,8.54,Drug or Biological
3,3,1,4389,NaN,TSAI,Covered,NaN,NaN,No,NaN,NaN,1,Allopathic & Osteopathic Physicians|Internal M...,KY,NaN,NJ,40205,NaN,NaN,0006027782,NaN,LOUISVILLE,NaN,NaN,1,NaN,08/01/2013,UNITED STATES,No Third Party Payment,Food and Beverage,KY,United States,No,45,NaN,NaN,JANUVIA,train,0006008182,YIYI,NaN,Merck Sharp & Dohme Corporation,NaN,In-kind items and services,1,4.64,Drug or Biological
4,4,1,559,NaN,REARDON,Covered,NaN,NaN,No,NaN,NaN,197,Allopathic & Osteopathic Physicians|Internal M...,RI,NaN,NY,02888-3655,NaN,NaN,0071101268,NaN,WARWICK,NaN,NaN,1,NaN,08/01/2013,UNITED STATES,No Third Party Payment,Food and Beverage,RI,United States,NaN,55,NaN,NaN,LYRICA,train,0025151501,EDWARD,NaN,Pfizer Inc.,NaN,In-kind items and services,1,13.80,Drug or Biological
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2697376,899122,3,1499,NaN,GROW,Yes,NaN,NaN,No,Gastroenterology,Gastroenterology,1268,Allopathic & Osteopathic Physicians|Internal M...,GA,NaN,FL,30601-3806,NaN,NaN,67546-111-12,Gastroenterology,ATHENS,NaN,NaN,1,NaN,12/31/2019,UNITED STATES,No Third Party Payment,Food and Beverage,GA,United States,NaN,1402232,NaN,67546-212-21,Alinia,train,67546-111-14,KELLY,NaN,"Romark Laboratories, LC",C,Cash or cash equivalent,3,16.04,Drug
2697377,899123,3,2948,NaN,BRAINARD,Yes,NaN,NaN,No,Gastroenterology,Gastroenterology,1268,Allopathic & Osteopathic Physicians|Family Med...,FL,NaN,FL,34202-3710,NaN,NaN,67546-111-12,Gastroenterology,BRADENTON,NaN,NH,1,NaN,12/31/2019,UNITED STATES,No Third Party Payment,Food and Beverage,FL,United States,NaN,1402238,NaN,67546-212-21,Alinia,train,67546-111-14,KAREN,NM,"Romark Laboratories, LC",O,Cash or cash equivalent,3,18.37,Drug
2697378,899124,3,4288,NaN,LOH,Yes,NaN,NaN,No,Human Tissue Products (361 HCT/P),Medical Device (510K),1234,Allopathic & Osteopathic Physicians|Ophthalmology,FL,NaN,FL,33143-3746,NaN,NaN,NaN,Human Tissue Products (361 HCT/P),MIAMI,NaN,IN,1,NaN,12/31/2019,UNITED STATES,No Third Party Payment,Food and Beverage,FL,United States,No,1402243,NaN

In [42]:
# df_scrap_1=pd.wide_to_long(df_scrap, stubnames='Product_Name_', i=['Record_ID'],j='Product_Name_Number').reset_index()
asd=df_scrap_1.query('Record_ID==2696')
asd[['Product_Name_Number','Product_Type_Number','Product_Name_','Product_Type_']]

,Product_Name_Number,Product_Type_Number,Product_Name_,Product_Type_
522,1,1,Evicel,Drug or Biological
300231,2,1,Biosurgicals,Drug or Biological
599940,3,1,NaN,Drug or Biological
899649,1,2,Evicel,Device or Medical Supply
1199358,2,2,Biosurgicals,Device or Medical Supply
1499067,3,2,NaN,Device or Medical Supply
1798776,1,3,Evicel,NaN
2098485,2,3,Biosurgicals,NaN
2398194,3,3,NaN,NaN


In [44]:
# df.query('Product_Name_1!=Product_Name_2 and Product_Name_3!=Product_Name_2 and Product_Type_1!=Product_Type_2 and Product_Type_2!=Product_Type_3 and Product_Name_2.notnull() and Product_Type_2.notnull()')
df.query('Record_ID==2696')

,Record_ID,Physician_ID,Company_ID,Total_Amount_of_Payment_USDollars,Date,Number_of_Payments,Form_of_Payment_or_Transfer_of_Value,Nature_of_Payment_or_Transfer_of_Value,City_of_Travel,State_of_Travel,Country_of_Travel,Ownership_Indicator,Third_Party_Recipient,Charity,Third_Party_Covered,Contextual_Information,Related_Product_Indicator,Product_Code_1,Product_Code_2,Product_Code_3,Product_Type_1,Product_Type_2,Product_Type_3,Product_Name_1,Product_Name_2,Product_Name_3,Product_Category_1,Product_Category_2,Product_Category_3,set,First_Name,Middle_Name,Last_Name,Name_Suffix,City,State_Phys,Zipcode,Country_Phys,Province,Primary_Specialty,License_State_1,License_State_2,License_State_3,License_State_4,License_State_5,Company_Name,State_Comp,Country_Comp
2695,2696,3663,57,31.98,08/06/2013,1,In-kind items and services,Food and Beverage,NaN,NaN,NaN,No,No Third Party Payment,No,NaN,NaN,Covered,6731339011,NaN,NaN,Drug or Biological,Device or Medical Supply,NaN,Evicel,Biosurgicals,NaN,NaN,NaN,NaN,train,WENDY,NaN,DAVIS,NaN,AUGUSTA,GA,30904,UNITED STATES,NaN,Allopathic & Osteopathic Physicians|Surgery,FL,GA,NM,NaN,NaN,Ethicon Inc.,NJ,United States


In [ ]:



df = df.astype(str)
for column in df.columns:
     if(df[column].dtype=='object'):
        df[column] = df[column].str.lower()

#Convert to Date
df['Date']=pd.to_datetime(df['Date'])

#type conv after str conv
df[['Company_ID','Number_of_Payments']]=df[['Company_ID','Number_of_Payments']].astype(int)
df['Total_Amount_of_Payment_USDollars']=df['Total_Amount_of_Payment_USDollars'].astype(float)


In [ ]:
###DROPPING RECORDS
df_scrap=df.query('Product_Code_1!="nan" and Product_Name_1=="nan" and set!="test"', engine='python')
df_scrap.Ownership_Indicator.value_counts()
df=pd.concat([df, df_scrap]).drop_duplicates(keep=False)

df_scrap=df.query('Product_Code_2!="nan" and Product_Name_2=="nan" and set!="test"', engine='python')
df_scrap.Ownership_Indicator.value_counts()
df=pd.concat([df, df_scrap]).drop_duplicates(keep=False)

df_scrap=df.query('Product_Code_3!="nan" and Product_Name_3=="nan" and set!="test"', engine='python')
df_scrap.Ownership_Indicator.value_counts()
df=pd.concat([df, df_scrap]).drop_duplicates(keep=False)



In [ ]:
###ENCODING CELL

name_list = pd.unique(df[['Product_Name_1', 'Product_Name_2', 'Product_Name_3']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Product_Name_1'] = df['Product_Name_1'].apply(product_encode)
df['Product_Name_2'] = df['Product_Name_2'].apply(product_encode)
df['Product_Name_3'] = df['Product_Name_3'].apply(product_encode)

name_list = pd.unique(df[['Product_Code_1', 'Product_Code_2', 'Product_Code_3']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Product_Code_1'] = df['Product_Code_1'].apply(product_encode)
df['Product_Code_2'] = df['Product_Code_2'].apply(product_encode)
df['Product_Code_3'] = df['Product_Code_3'].apply(product_encode)

name_list = pd.unique(df[['Product_Category_1', 'Product_Category_2', 'Product_Category_3']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Product_Category_1'] = df['Product_Category_1'].apply(product_encode)
df['Product_Category_2'] = df['Product_Category_2'].apply(product_encode)
df['Product_Category_3'] = df['Product_Category_3'].apply(product_encode)

name_list = pd.unique(df[['Product_Type_1', 'Product_Type_2','Product_Type_3']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Product_Type_1'] = df['Product_Type_1'].apply(product_encode)
df['Product_Type_2'] = df['Product_Type_2'].apply(product_encode)
df['Product_Type_3'] = df['Product_Type_3'].apply(product_encode)


name_list = pd.unique(df[['License_State_1', 'License_State_2', 'License_State_3', 'License_State_4', 'License_State_5']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def license_encode(license_state):
    return name_list_dict.get(license_state)

df['License_State_1']=df['License_State_1'].apply(license_encode)
df['License_State_2']=df['License_State_2'].apply(license_encode)
df['License_State_3']=df['License_State_3'].apply(license_encode)
df['License_State_4']=df['License_State_4'].apply(license_encode)
df['License_State_5']=df['License_State_5'].apply(license_encode)

name_list = pd.unique(df[['Country_Comp', 'Country_Phys','Country_of_Travel']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['Country_Comp'] = df['Country_Comp'].apply(product_encode)
df['Country_Phys'] = df['Country_Phys'].apply(product_encode)
df['Country_of_Travel'] = df['Country_of_Travel'].apply(product_encode)

name_list = pd.unique(df[['State_Comp', 'State_Phys', 'State_of_Travel']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['State_Comp'] = df['State_Comp'].apply(product_encode)
df['State_Phys'] = df['State_Phys'].apply(product_encode)
df['State_of_Travel'] = df['State_of_Travel'].apply(product_encode)

name_list = pd.unique(df[['City_of_Travel', 'City']].values.ravel('K'))
# name_list = np.delete(name_list,np.where(name_list=="nan"))
name_list_dict = {}
int_code=int(1)
for name in name_list:
    name_list_dict.update({name:int_code})
    int_code+=1

def product_encode(product_name):
    return name_list_dict.get(product_name)

df['City_of_Travel'] = df['City_of_Travel'].apply(product_encode)
df['City'] = df['City'].apply(product_encode)

df['Ownership_Indicator']=np.where((df.Ownership_Indicator == "yes"),1,df.Ownership_Indicator)
df['Ownership_Indicator']=np.where((df.Ownership_Indicator == "no"),0,df.Ownership_Indicator)
df['Ownership_Indicator']=df['Ownership_Indicator'].astype(int)

df['Date_year']=df['Date'].dt.year
df['Date_month']=df['Date'].dt.month
df['Date_day']=df['Date'].dt.day

columns_encode=['Nature_of_Payment_or_Transfer_of_Value','Form_of_Payment_or_Transfer_of_Value','Zipcode','Third_Party_Recipient', 'Charity', 'Third_Party_Covered','Contextual_Information', 'Related_Product_Indicator','Primary_Specialty']
for column in columns_encode:
    df[column]=df[column].astype('category')
    df[column]=df[column].cat.codes


In [ ]:
###DROPPING COLUMNS
df_safe = df
df.drop(columns=['Date','Record_ID', 'First_Name', 'Middle_Name', 'Last_Name', 'Name_Suffix', 'Company_Name','Province'],inplace=True)
# df.drop(columns=['Date','Record_ID', 'First_Name', 'Middle_Name', 'Last_Name', 'Name_Suffix', 'Company_Name','Province','Product_Type_1','Product_Type_2','Product_Type_3','Product_Code_1','Product_Code_2','Product_Code_3'],inplace=True)

###SPLIT
df_test=df[df.set=="test"]
df_train=df[df.set=="train"]

###drop set column
df_test.drop(columns=['set'],inplace=True)
df_train.drop(columns=['set'],inplace=True)

##send predicted attribute to end
first_col = df_train.pop("Ownership_Indicator")
df_train.insert((df_train.shape[1]),"Ownership_Indicator",first_col)

first_col = df_test.pop("Ownership_Indicator")
df_test.insert((df_test.shape[1]),"Ownership_Indicator",first_col)

##send Physician ID to end
first_col = df_train.pop("Physician_ID")
df_train.insert((df_train.shape[1]),"Physician_ID",first_col)

first_col = df_test.pop("Physician_ID")
df_test.insert((df_test.shape[1]),"Physician_ID",first_col)



In [ ]:
df_test

In [ ]:
##heatmap cell
fig = plt.figure(figsize = (30,30))
ax = fig.gca()

heatmap = sns.heatmap(df_train[df_train.columns[0:41]].corr(), annot=True)
fig = heatmap.get_figure()
fig.savefig("heatmap_after_na_encode_OI.png")

In [ ]:
###downsampling
df_train_maj=df_train[df_train.Ownership_Indicator==0]
df_train_min=df_train[df_train.Ownership_Indicator==1]
df_train_maj_down=resample(df_train_maj, replace=False, n_samples=df_train_min.shape[0], random_state=2021)
df_train_final = pd.concat([df_train_maj_down, df_train_min])

###test train split of own train set
df_train_final['is_train']=np.random.uniform(0,1,len(df_train_final)) <=.80
train, test = df_train_final[df_train_final['is_train']==True], df_train_final[df_train_final['is_train']==False]
train = shuffle(train)
test = shuffle(test)

In [ ]:
###feature and label split
features=train[train.columns[0:(train.shape[1]-3)]]
y=train['Ownership_Indicator']
x_test = test[test.columns[0:(test.shape[1]-3)]]
y_test = test['Ownership_Indicator']

###model training
# clf = make_pipeline(preprocessing.StandardScaler(), SVC(gamma='auto',kernel='rbf'))
# clf = make_pipeline(preprocessing.StandardScaler(), LinearSVC(random_state=2021, tol=1e-5))
clf = RandomForestClassifier(n_estimators = 150, n_jobs=-1, oob_score = True, random_state=2021)
clf.fit(features, y)

# Accuracy Scores
print ('Internal Accuracy Score', clf.oob_score_)
print ('accuracy: TRAINING', clf.score(features,y))
print ('accuracy: TESTING', clf.score(x_test,y_test))
score_test=clf.predict(x_test)
# print(score_test.shape)

In [ ]:
print(test.shape)
print(train.shape)

In [ ]:
###Creating Dataframes to find accuracy of Physician Level

finalAccPhysID=test['Physician_ID']
finalAcc={'Physician_ID':finalAccPhysID, 'Score':score_test}
finalAccDF=pd.DataFrame(data=finalAcc)
# finalAccDF['Physician_ID'].unique().size
physUFAD=finalAccDF['Physician_ID'].unique()
sc=[None]*physUFAD.size
for i in range(physUFAD.size):
    physician=physUFAD[i]
    if(finalAccDF.query('Physician_ID==@physician and Score==1').shape[0]>0):
        sc[i]=1
    else:
        sc[i]=0
        
finTestPredict={'Physician_ID':physUFAD, 'Score':sc}
finTestPredictDF=pd.DataFrame(data=finTestPredict)

TrueFinalAcc={'Physician_ID':finalAccPhysID, 'Score':test['Ownership_Indicator']}
TrueFinalAccDF=pd.DataFrame(data=TrueFinalAcc)
# finalAccDF['Physician_ID'].unique().size
physUFAD=TrueFinalAccDF['Physician_ID'].unique()
sc=[None]*physUFAD.size
for i in range(physUFAD.size):
    physician=physUFAD[i]
    if(TrueFinalAccDF.query('Physician_ID==@physician and Score==1').shape[0]>0):
        sc[i]=1
    else:
        sc[i]=0

TrueFinTest={'Physician_ID':physUFAD, 'Score':sc}
TrueFinTestDF=pd.DataFrame(data=TrueFinTest)




In [ ]:
###Finding Accuracy
ownTestScore=TrueFinTestDF['Score']-finTestPredictDF['Score']
print(ownTestScore)
accuracy=(ownTestScore.value_counts(dropna=False,sort=False,ascending=True).loc[0])/ownTestScore.size*100
print(accuracy)

In [ ]:
df_test
df_test.Physician_ID.unique().size

In [ ]:
print(features.columns)
print(df_test.columns)
predict_set=df_test
predict_set.drop(columns=['Ownership_Indicator','Physician_ID'],inplace=True)
print(features.shape)
print(predict_set.shape)

In [ ]:
score = clf.predict(predict_set)

In [ ]:
score.size

In [ ]:
phys_id_list=df_main_test['Physician_ID']

In [ ]:
submit={'Physician_ID':phys_id_list, 'Score':score}
submit_df=pd.DataFrame(data=submit)
submit_df['Physician_ID'].unique().size
physUnique=submit_df['Physician_ID'].unique()
physUnique.size
physUnique[0]

In [ ]:

sc=[None]*1000
for i in range(physUnique.size):
    physician=physUnique[i]
    if(submit_df.query('Physician_ID==@physician and Score==1').shape[0]>0):
        sc[i]=1
    else:
        sc[i]=0
        

In [ ]:
final_submit={'id':physUnique, 'prediction':sc}
fs_df=pd.DataFrame(data=final_submit)
fs_df

In [ ]:
fs_df.to_csv("./submission2.csv",index=False)

In [ ]:
print(physUnique)
print(sc)

In [ ]:
inspect1 = pd.read_csv("./submission1.csv")
dfscrap=fs_df['prediction']-inspect1['prediction']
dfscrap.value_counts()